In [1]:
!pip install keras==3.0.0 --upgrade --quiet

import os
os.environ["KERAS_BACKEND"] = "torch"

import torch
import keras
print(keras.__version__)

3.0.0


## An LSTM Autoencoder

In [2]:
# Hyperparameters
n_in = 9
learning_rate = 1e-3
num_epoch = 300

In [7]:
class LSTMAE(keras.Model):
    def __init__(self):
        super().__init__()
        self.lstm1 = keras.layers.LSTM(100, activation='relu',
                                       input_shape=(n_in,1), return_sequences=True)
        # Can we only use return_sequence in the last LSTM encoder layer
        # and don’t use RepeatVector before the first LSTM decoder layer?
        # self.repeatvector = keras.layers.RepeatVector(n_in)
        self.lstm2 = keras.layers.LSTM(100, activation='relu',
                                       return_sequences=True)
        self.compress = keras.layers.TimeDistributed(keras.layers.Dense(1))

    def encode(self, x):
        # return self.repeatvector(self.lstm1(x))
        return self.lstm1(x)

    def decode(self, z):
        return self.compress(self.lstm2(z))

    def call(self, x, training=False):
        return self.decode(self.encode(x))

In [8]:
from numpy import array

sequence = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
sequence = sequence.reshape((1, n_in, 1))

model = LSTMAE()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.compile(optimizer='adam', loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
model.fit(sequence, sequence, epochs=300, verbose=0)

In [10]:
# demonstrate recreation
yhat = model.predict(sequence, verbose=0)
print(yhat[0,:,0])

[0.11535606 0.20271727 0.29728785 0.39617553 0.49765027 0.60010123
 0.70199287 0.8019005  0.89855   ]
